# 神经网络的学习

In [2]:
import numpy as np

### 均方误差

In [3]:
def mean_squared_error(y, t):
    return .5 * np.sum((y-t) ** 2)

In [7]:
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
mean_squared_error(y, t)

0.09750000000000003

In [8]:
y = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
mean_squared_error(y, t)

0.5975

### 交叉熵误差

In [10]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return - np.sum(t * np.log(y + delta))
# 作为保护性对策，添加一个微小值可以防止负无限大的发生。

In [11]:
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
cross_entropy_error(y, t)

0.510825457099338

In [13]:
y = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
cross_entropy_error(y, t)

2.302584092994546

### mini-batch学习

In [14]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

In [15]:
(X_train, t_train), (X_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
X_train.shape, t_train.shape

((60000, 784), (60000, 10))

In [16]:
train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
X_batch = X_train[batch_mask]
t_batch = t_train[batch_mask]

### mini-batch版交叉熵误差的实现

In [17]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+1e-7)) / batch_size

In [20]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

因为`t`中标签是以 `[2, 7, 0, 9, 4]` 的形式存储的，所以 `y[np.arange(batch_size), t]`能抽出各个数据的正确解标签对应的神经网络的输出。

## 数值微分

In [81]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [8]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    for idx in range(x.size):
        origin_val = x[idx]
        x[idx] = origin_val + h
        fxh1 = f(x)
        
        x[idx] = origin_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = origin_val
    return grad

In [84]:
# 必须写成带小数点的形式，或者显式指定dtype=float64
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([6., 8.])

In [85]:
numerical_gradient(function_2, np.array([0.0, 2.0]))

array([0., 4.])

In [86]:
numerical_gradient(function_2, np.array([3.0, 0.0]))

array([6., 0.])

## 梯度

### 梯度法

In [95]:
def gradient_descent(f, init_x, lr=.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr*grad
    return x

In [115]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, lr=.1)

array([-6.11110793e-10,  8.14814391e-10])

学习率过大或者过小都无法得到好的结果。(注意：`init_x` 会被直接修改)

In [123]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, lr=10)

array([-2.58983747e+13, -1.29524862e+12])

In [125]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x, lr=.000001)

array([-2.99940006,  3.99920008])

实验结果表明，学习率过大的话，会发散成一个很大的值；反过来，学习率过小的话，基本上没怎么更新就结束了。也就是说，设定合适的学习率是一个很重要的问题。

### 神经网络的梯度

In [2]:
from common.function import softmax
from common.gradient import numerical_gradient
class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [145]:
net = SimpleNet()
net.W

array([[ 1.18678376,  0.16229639, -0.74186488],
       [ 1.17970482,  0.2645372 , -0.43377498]])

In [141]:
x = np.array([.6, .9])
p = net.predict(x)
p

array([0.47183382, 0.88341895, 0.82781457])

In [142]:
t = np.array([0, 0, 1])
net.loss(x, t)

0.9587798366580182

In [147]:
def f(W):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)
dW

array([[ 0.45769878, -0.49137832,  0.03367953],
       [ 0.68654818, -0.73706747,  0.0505193 ]])

## 学习算法的实现

### 2层神经网络的类

In [12]:
from two_layer_net import TwoLayerNet
import numpy as np

In [13]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape, net.params['b1'].shape, net.params['W2'].shape, net.params['b2'].shape

((784, 100), (100,), (100, 10), (10,))

In [14]:
x = np.random.rand(100, 784) # 假的训练数据
t = np.random.rand(100, 10) # 假的测试标签
y = net.predict(x)

In [10]:
grads = net.numerical_gradient(x, t)

In [11]:
grads['W1'].shape, grads['b1'].shape, grads['W2'].shape, grads['b2'].shape

((784, 100), (100,), (100, 10), (10,))